In [142]:
import pandas as pd

In [143]:
import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Github/Patricionog/Modulio')
import Foragio as fg
import Excelio as ex
import Framio as fr
import Stringio as st
import Listio as ls

In [144]:
# Nombres de archivos.

Filename_Migraciones = 'Migraciones'
Filename_Sedes_Basico = 'Sedes_Basico'
Filename_Sedes = 'Sedes'
Filename_Secciones = 'Secciones'

In [145]:
# Carpeta contenedora.

Path = 'C:/Users/tomas/Documents/Programación/Github/Patricionog/Laboratorio de datos/Práctico 1/'

In [146]:
# Dataframes.

Migraciones = pd.read_csv(Path + Filename_Migraciones + '.csv')
Sedes_Basico = pd.read_csv(Path + Filename_Sedes_Basico + '.csv')
Sedes = pd.read_csv(Path + Filename_Sedes + '.csv')
Secciones = pd.read_csv(Path + Filename_Secciones + '.csv')

In [147]:
# Migraciones

Migraciones.head()

,Country Origin Name,Country Origin Code,Migration by Gender Name,Migration by Gender Code,Country Dest Name,Country Dest Code,1960 [1960],1970 [1970],1980 [1980],1990 [1990],2000 [2000]
0,Afghanistan,AFG,Female,FEM,Afghanistan,AFG,0,0,0,0,0
1,Afghanistan,AFG,Female,FEM,Albania,ALB,0,0,0,0,0
2,Afghanistan,AFG,Female,FEM,Algeria,DZA,22,8,6,5,4
3,Afghanistan,AFG,Female,FEM,American Samoa,ASM,0,0,0,0,0
4,Afghanistan,AFG,Female,FEM,Andorra,AND,0,0,0,2,3


In [148]:
# Quitar filas con el genero TOT, ya que son la suma de FEM y MAL.
Migraciones = fr.Drop_Selected_Rows(Migraciones, 'Migration by Gender Code', 'TOT', Condition = 'Match')

In [149]:
# Quitar columnas que no importan.

Columnas = ['Country Origin Code',	'Country Origin Name', 'Country Dest Code', 'Country Dest Name', '2000 [2000]']

Migraciones = Migraciones[Columnas]

In [150]:
# Cambiar nombres de columnas por unos más descriptivos.

New_Names = {'Country Dest Name': 'Country Destiny Name',
             'Country Dest Code': 'Country Destiny Id',
             'Country Origin Code': 'Country Origin Id',
             '2000 [2000]': 'migrants_on_year_2000'}

Migraciones = fr.Change_Column_Names_By_Dictionary(Migraciones, New_Names)

Migraciones.head()

,Country Origin Id,Country Origin Name,Country Destiny Id,Country Destiny Name,migrants_on_year_2000
0,AFG,Afghanistan,AFG,Afghanistan,0
1,AFG,Afghanistan,ALB,Albania,0
2,AFG,Afghanistan,DZA,Algeria,4
3,AFG,Afghanistan,ASM,American Samoa,0
4,AFG,Afghanistan,AND,Andorra,3


In [151]:
# Cambiar estilo de los nombres de las columnas.

Migraciones = fr.Casing_Column_Names(Migraciones, "Snake Case")

Migraciones.head()

,country_origin_id,country_origin_name,country_destiny_id,country_destiny_name,migrants_on_year_2000
0,AFG,Afghanistan,AFG,Afghanistan,0
1,AFG,Afghanistan,ALB,Albania,0
2,AFG,Afghanistan,DZA,Algeria,4
3,AFG,Afghanistan,ASM,American Samoa,0
4,AFG,Afghanistan,AND,Andorra,3


#### Problema. 
- Los países son a veces "subpaíses", como Taiwán o Hong Kong. Si se saca el país padre (China) se pierde info, pero si solo se deja al padre tampoco tiene mucho sentido, porque no es China China.
¿Se podría armar otra tabla indicando eso, con hijos y padres? No sé.

In [152]:
# Seleccionamos filas con problemas.
Migraciones_Problems = fr.Get_Selected_Rows_By_Column(Migraciones, 'country_destiny_name', ',', Condition = 'Contains')

In [153]:
# Vemos los valores únicos de esos problemas.
Migraciones_Problems['country_destiny_name'].unique()

array(['Bahamas, The', 'Congo, Dem. Rep.', 'Congo, Rep.',
       'Egypt, Arab Rep.', 'Gambia, The', 'Hong Kong SAR, China',
       'Iran, Islamic Rep.', 'Korea, Dem. Rep.', 'Korea, Rep.',
       'Macao SAR, China', 'Macedonia, FYR', 'Micronesia, Fed. Sts.',
       'Taiwan, China', 'Venezuela, RB', 'Virgin Islands, British',
       'Yemen, Rep.'], dtype=object)

In [154]:
# Casos especiales.

New_Names = {'Congo, Dem. Rep.': 'Democratic Republic of the Congo',
             'Congo, Rep.': 'Republic of the Congo',
             'Korea, Dem. Rep.': 'North Korea',
             'Korea, Rep.': 'South Korea'}

for Old_Name, New_Name in New_Names.items():
    Migraciones['country_destiny_name'] = Migraciones['country_destiny_name'].replace(Old_Name, New_Name)
    Migraciones['country_origin_name'] = Migraciones['country_origin_name'].replace(Old_Name, New_Name)

In [155]:
# Removemos lo que hay de string después de la coma en las dos columnas de nombres de países.

Migraciones = fr.Apply_Operations_To_Selected_Rows(Migraciones,
                                     Filtered_Column = 'country_destiny_name',
                                     Filter_Value = ',',
                                     Condition ='Contains',
                                     Columns_To_Operate = ['country_destiny_name'],
                                     Operations = [lambda x: ls.Slice_Iterable_By_Reference(x, ',', Remove='After', Include = False, Appears=1)])

Migraciones = fr.Apply_Operations_To_Selected_Rows(Migraciones,
                                     Filtered_Column = 'country_origin_name',
                                     Filter_Value = ',',
                                     Condition = 'Contains',
                                     Columns_To_Operate = ['country_origin_name'],
                                     Operations = [lambda x: ls.Slice_Iterable_By_Reference(x, ',', Remove='After', Include = False, Appears=1)])

In [156]:
# Quitar algunas palabras molestas o que pueden causar confusión.

Migraciones = Migraciones.replace('Macao SAR', 'Macao')
Migraciones = Migraciones.replace('Hong Kong SAR', 'Hong Kong')
Migraciones = Migraciones.replace('Virgin Islands (U.S.)', 'Virgin Islands')
Migraciones = Migraciones.replace(f"Cote d'Ivoire", 'Cote dIvoire')
Migraciones = Migraciones.replace(f"Falkland Islands (Malvinas)", 'Malvinas')
Migraciones = Migraciones.replace(f"Guinea-Bissau", 'Guinea Bissau')

Migraciones[Migraciones['country_destiny_name'] == 'Macao'].head()

,country_origin_id,country_origin_name,country_destiny_id,country_destiny_name,migrants_on_year_2000
119,AFG,Afghanistan,MAC,Macao,2
350,AFG,Afghanistan,MAC,Macao,2
581,ALB,Albania,MAC,Macao,0
812,ALB,Albania,MAC,Macao,0
1043,DZA,Algeria,MAC,Macao,0


In [157]:
# Verificación.
Migraciones[Migraciones['country_origin_name'] == 'Democratic Republic of the Congo'].head()

,country_origin_id,country_origin_name,country_destiny_id,country_destiny_name,migrants_on_year_2000
20328,COD,Democratic Republic of the Congo,AFG,Afghanistan,5
20329,COD,Democratic Republic of the Congo,ALB,Albania,0
20330,COD,Democratic Republic of the Congo,DZA,Algeria,0
20331,COD,Democratic Republic of the Congo,ASM,American Samoa,0
20332,COD,Democratic Republic of the Congo,AND,Andorra,6


In [158]:
# Convertir a numéricos los migrantes y borrar los strings.
Migraciones['migrants_on_year_2000'] = pd.to_numeric(Migraciones['migrants_on_year_2000'], errors='coerce')

In [159]:
# Chequear NULLS.

Count_Nulls_By_Columns = Migraciones.isnull().sum()

Count_Nulls_By_Columns

country_origin_id           0
country_origin_name         0
country_destiny_id          0
country_destiny_name        0
migrants_on_year_2000    5032
dtype: int64

In [160]:
# Llenar NULLS con 0.
Migraciones['migrants_on_year_2000'] = Migraciones['migrants_on_year_2000'].fillna(0)

In [161]:
# Guardar archivo.
Migraciones.to_csv('Migraciones_Modificado.csv', index=False)